In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from skimage.util import random_noise
from skimage.filters import threshold_local
import os
from PIL import Image
import cv2

In [ ]:
mnist = fetch_openml('mnist_784')
X, y = mnist['data'].to_numpy(), mnist['target'].to_numpy()

num_samples, num_features = X.shape

num_train = 60000
X_train, X_test = X[:num_train], X[num_train:]
y_train, y_test = y[:num_train], y[num_train:]

/home/gyuri/.venv/venv/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## Zaj hozzáadása az adathalmazhoz

In [ ]:
noise_level = 2

X_train_noisy = np.array([random_noise(image, mode='gaussian', var=noise_level).flatten() for image in X_train])
X_test_noisy = np.array([random_noise(image, mode='gaussian', var=noise_level).flatten() for image in X_test])

model = MLPClassifier(solver='adam', activation='tanh', hidden_layer_sizes=(64,64),
                    max_iter=1, learning_rate='adaptive')
model.fit(X_train_noisy, y_train)

y_pred = model.predict(X_test_noisy)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

classes = np.unique(y_test)
plt.figure(figsize=(12, 12))
for i, cls in enumerate(classes):
    plt.subplot(5, 5, i + 1)
    idx = np.where(y_test == cls)[0][0]
    plt.imshow(X_test_noisy[idx].reshape(28, 28), cmap='gray')
    plt.title(f'Osztály: {cls}')
    plt.axis('off')
plt.show()


In [ ]:
def image_to_bin(img):
    data = list(img.getdata())
    for i in range(len(data)):
        data[i] = 255 - data[i]
    data = np.array(data)/256.0
    return data

# Gimpben rajzolt számok

In [ ]:
image_folder = "drawn_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = model.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Kézzel írott számok

In [ ]:
image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = model.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Küszöbölés invertálással

In [ ]:
def local_threshold_mean(image, kernel_size):
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    convolved_image = cv2.filter2D(image, -1, np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size * kernel_size))
    thresholded_image = np.where(image > convolved_image, 255, 0).astype(np.uint8)
    return thresholded_image

def invert_colors(img):
    return cv2.bitwise_not(img)

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) 
        binimg = local_threshold_mean(image, kernel_size=25)
        inverted_img = invert_colors(binimg)
        p = model.predict([inverted_img.flatten()])
        images.append(inverted_img)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Kontúrkövetéssel

In [ ]:
def contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (255,0, 255), 1)
    return contour_img

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        image_with_contours = contours(cv2.cvtColor(image, cv2.COLOR_GRAY2RGB))
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = model.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Észrevételek

- Kisebb modell pontosság
- Rövidebb tanulási idő
- Jobb becslés a feldolgozatlan képeken
- Feldolgozott képek felismerése hibásabb